# Static traffic assignment iPython code
*** 
- [Section 1](#graph) upload the graph
- [Section 2](#demand) upload the demand
- [Section 3](#assignment) encode the all or nothing function
- TO DO

***
We want to solve the following problem (TAP):
\begin{align}
\min_{h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\end{align}

TO DO: EXPLAIN THE INTUITION BEHIND THE STATIC TRAFFIC ASSIGNMENT.

## 0. Import some libraries useful for the project
***

In [13]:
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt

## 1. Import the road network and its characteristics <a id="graph"></a>
***
The road network is composed by:
- $G=(E,V)$ a **strongly connected oriented graph** with vertices $v\in V$ and edges $e\in E$
- $t_{0,e}\in\mathbb{R}_+$ the **free flow travel time** of the edge $e$, $\forall e\in E$
- $c_{e}\in\mathbb{R}_+$ the **flow capacity** of the edge $e$, $\forall e\in E$
- $B_e\in\mathbb{R}_+$ and $p_e\in\mathbb{N}_+$ two characteristics of the edge $e$ useful for estimating the travel time of vehicle on the edge $e$ given the flow of the edge $e$, $\forall e\in E$

Data are provided from https://github.com/bstabler/TransportationNetworks.
They are is the folder `data/`.

The road network and its characteristics are read in the file `network.py` with the function:
- `init_travel_time_function(network) : graph_wrapped`: takes as parameter a string which represent the path to the data file TNTP `network` and return in `graph_wrapped` the table that can be used to compute the travel time of every links, and the index of the free flow travel time, of the B, of the capacity and of the power columns.
_TO DO: Better explain this_

In [6]:
from network import init_travel_time_function

## 2. Import the road demand <a id="demand"></a>
***
The road demand is a matrix $r \in \mathbb{R}_+^{|V| \times |V|}$ where:
- $r_{o,d}$ is the demand between the origin $o\in V$ and the destination $d\in V$.

Remark that for most of the vertices $u,v\in V$, $r_{u,v}=0$ so the demand matrix is very sparse.

The road demand are read in the file `network.py` with the function:
- `read_demand(file)`: takes as parameter a string which represent the path to the data file TNTP `network` and return in `demand` a dictionary of keys based sparse matrix representing the demand $r$.

In [ ]:
from network import read_demand

## 3. Define the link travel time as a function of the link flow <a id="travel_time"></a>
***
The travel time of every link $e\in E$ is given as a function of the flow on the link $f_e\in\mathbb{R}_+$ and of [the characteristics of the edge](#graph):
$$ t_e(f_e) = t_{0,e} \cdot \left( 1 + B_e \cdot \left(\frac{f_e}{c_e}\right)^{p_e} \right) $$
This is the classical BPR function as defined in [[Patriksson](#references)]. 

The function `travel_time(flow, graph_wrapped): travel_time` computes it: it gives the travel time vector for every link given the [graph_wrapped](#graph) and the vector of link flow.

A test is provided at the [end of the notebook](#tests).

In [14]:
def travel_time(flow, graph_wrapped):
    """
    Parameter: 
        flow: a vector that represents the flow of every links
        graph_wrapped: should be = (table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term)
    Return:
        A travel time vector for every links given the flow allocation flow.
        
        The travel time of a link (one row of the table) is t(f) = t0 * (1 + B*(f/capacity)**power)
    """
    table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term = graph_wrapped
    return table_net[:,index_fft] * (1 + table_net[:,index_B]*(flow/table_net[:,index_capacity])**table_net[:,index_power])

## 4. Assignment <a id="assignment"></a>
***
TO BE CHANGED AND COMMENTED AS ABOVE.
- PUT get_graph IN network.py. THEN IMPORT IT AND EXPLAIN WHY WE NEED IT.
- EXPLAIN initialization

In [ ]:
def get_graph(graph_wrapped):
    """
    Parameter:
        graph_wrapped: should be = (table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term)
    Return:
        A dictionary encoding the graph: graph[origin][destination] = link_id
    """
    table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term = graph_wrapped
    
    graph = {}
    for i in range(table_net.shape[0]):
        link_index = i
        node_init = int(table_net[i][index_init])
        node_term = int(table_net[i][index_term])
        
        if node_init not in graph:
            graph[node_init] = {}
        # if node_term not in graph[node_init]:
        #     graph[node_init][node_term] = {}
        # we assume that there is only one possible directed link between two nodes
        graph[node_init][node_term] = link_index
    return graph

def initialization(network):
    """
    Parameter:
        network: the file name of the data 
    Return:
        graph_wrapped, demand, graph, nb_links using previously defined functions
    """
    graph_wrapped = init_travel_time_function(network)
    demand = read_demand(network)
    graph = get_graph(graph_wrapped)

    table_net, _, _, _, _, _, _ = graph_wrapped
    nb_links = table_net.shape[0]
    return graph_wrapped, demand, graph, nb_links

# we have the demand in the sparse matrix demand, and the travel_time function in travel_time
# we have the graph of the network in graph


In [ ]:
def neighbours(node, adj):
    """
    Parameter:
        node: node for which to find neigbours for
        adj: adjacency matrix that describes network
    Return:
        List of nodes that are neighbours to node
    """
    return np.nonzero(adj[node])[0].tolist()

def add_flow_dijkstra(adj, src, target, faon, flow, g):
    """
    Parameter:
        adj: adjacency matrix that describes network
        src: starting node for OD pair
        target: end node for OD pair
        faon: current all or nothing flow allocation that the algorithm is building
        flow: demand of the current of OD pair
        g: graph dictionary, g[node_init] = {node_term_1: link_from_init_to_term_1, node_term_2: link_from_init_to_term_2}
    Return:
        Returns all or nothing flow allocation, where flow is allocated by shortest path for OD pair (as determined by Dijkstra's algorithm)
    """
    q = [i for i in range(len(adj))] # queue of nodes
    dist = [np.inf for i in range(len(adj))] # node distances
    prev = [None for i in range(len(adj))] # predecessor in shortest path
    
    dist[src] = 0 # set source node's distance to itself as zero
    
    while len(q) != 0:
        u = min(q, key=lambda n:dist[n])
        q.remove(u)
        
        if u == target: # when target is reach, use predecessors to allocate all or nothing flow to links in shortest path
            while u != src:
                prev_node = prev[u]
                curr_link = g[prev_node+1][u+1]
                faon[curr_link] += flow
                u = prev_node
            return faon
        
        for v in neighbours(u, adj):
            alt = dist[u] + adj[u][v]
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u



In [ ]:
import scipy
from scipy.sparse.csgraph import dijkstra

def update_travel_time_from_tt(graph, tt):
    nb_nodes = len(graph)
    G = np.zeros(shape=(nb_nodes,nb_nodes))
    G = scipy.sparse.lil_matrix(G)
    for i in graph.keys():
        for j in graph[i].keys():
            G[i-1, j-1] = tt[graph[i][j]] # THIS IS VERY BAD BECAUSE I PUT i-1 and j-1 instead of i and j
    G = G.tocsr()
    return G

def put_on_shortest_path(faon, o_tmp, d_tmp, flow_tmp, tt, g, G):
    """
    Parameter:
        faon: current all or nothing flow allocation that the algorithm is building
        o_tmp: origin of the current od pair
        d_tmp: destination of the current od pair
        flow_tmp: demand of the current of pair
        tt: travel time on every links, tt[l] = travel time of the link l
        g: graph dictionary, g[node_init] = {node_term_1: link_from_init_to_term_1, node_term_2: link_from_init_to_term_2}
    Return:
        TO WRITE
    """
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True) # change
    
    node_tmp = d_tmp
    # using the dijkstra, we build the fastest path and we put the flow on it.
    while node_tmp != o_tmp:
        node_tmp_d = return_predecessors[o_tmp][node_tmp]
        # Here we need the graph_dict to recover the link id from the nodes id.
        link_tmp = g[node_tmp_d+1][node_tmp+1] # this is very bad because we use node_tmp +1 and node_tmp_d + 1 instead of node_tmp and node_tmp_d
        faon[link_tmp] += flow_tmp
        node_tmp = node_tmp_d

    return faon

# all or nothing function
def all_or_nothing(d, tt, g):
    """
    Parameter:
        d: a demand as sparse matrix, d[origin][dest] = demand between origin and dest
        tt: travel time on every links, tt[l] = travel time of the link l
        g: graph dictionary, g[node_init] = {node_term_1: link_from_init_to_term_1, node_term_2: link_from_init_to_term_2}
    Return:
        The all or nothing flow allocation corresponding to the demand d and the travel time tt.
    """
    nb_links = tt.shape
    faon = np.zeros(nb_links)
    
    G = update_travel_time_from_tt(g, tt) # to remove for putting the dijkstra algorithm
    adj = G.toarray()
    
    for o_tmp, d_tmp in d.keys():
        #sp = find_sp(o_tmp, d_tmp)
        #faon = add_flow(faon, sp, d[o_tmp, d_tmp])
        faon = add_flow_dijkstra(adj, o_tmp, d_tmp, faon, d[o_tmp, d_tmp], g)
        # flow_tmp = d[o_tmp, d_tmp]
        # faon = put_on_shortest_path(faon, o_tmp, d_tmp, flow_tmp, tt, g, G) # to change when we write our own 
    return faon

#def find_sp(g):

def potential(f, graph_wrapped):
    """
    Parameter:
        flow: a vector that represents the flow of every links
        graph_wrapped: should be = (table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term)
    Return:
        The potential function tha t we want to minimize.
        
        The potential function is (one row of the table) is sum(int(t_e(s), s in [0, f_e]), e) = sum(t0 * (f + B*(f/capacity)**(power+1)/power)).    
        This function is useful for doing a line search, it computes the potential at flow assignment f.
    """
    table_net, index_fft, index_B, index_capacity, index_power, index_init, index_term = graph_wrapped
    pot_tmp = table_net[:,index_fft] * f * (1 + (table_net[:,index_B] / table_net[:,index_power])*(f/table_net[:,index_capacity])**table_net[:,index_power])
    return np.sum(pot_tmp)

def line_search(f, res=20):
    """
    Parameter:
        f: the function that we want to minimize between 0 and 1
        res: the resolution of the grid ([0, 1] is divided in 2^res points).
    Return:
        On a grid of 2^res points bw 0 and 1, find global minimum for a continuous convex function using bisection 
    """
    d = 1. / (2**res - 1) # d: Size of interval between adjacent points
    l, r = 0, 2**res - 1 # l: first point, r: last point
    while r - l > 1:
        if f(l * d) <= f(l * d + d): # Interval l: [l*d, l*d + d]
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

# write pseudocode STA
epsilon = 0.0000001

# initialization
network = 'Traffic_network_framework/data/SiouxFalls/SiouxFalls'
graph_wrapped, demand, graph, nb_links = initialization(network)

table_net, _, _, _, _, _, _ = graph_wrapped
nb_links = table_net.shape[0]


f = all_or_nothing(demand, travel_time(np.zeros(nb_links), graph_wrapped), graph)
f_iter = []
tt_iter = []

alpha = 1.0
i = 0
while alpha>epsilon and i<1000:
    f_iter.append(f)
    tt = travel_time(f, graph_wrapped)
    faon = all_or_nothing(demand, tt, graph)
    tt_iter.append(np.dot(tt,(f - faon)))
    alpha = line_search(lambda a : potential((1. - a) * f + a * faon, graph_wrapped))
    f = (1-alpha) * f + alpha * faon
    i = i+1
print(i)
print(f)

In [ ]:
flow, cost = flow_cost_solution(network)
print(flow)

In [ ]:
print(potential(flow, graph_wrapped))
print(potential(f, graph_wrapped))
print(np.linalg.norm(f-flow)/np.linalg.norm(flow))

In [ ]:
print("Graph")
print(graph)
print("Travel time")
print(travel_time(flow, graph_wrapped))
print("Demand")
print(demand)

In [ ]:
norm_flow_loss = [np.linalg.norm(f-flow,2) for f in f_iter]
plt.figure()
plt.title("Norm flow loss")
plt.ylabel("Norm loss")
plt.xlabel("No of iterations")
plt.plot(norm_flow_loss)
plt.show()

potential_loss = [potential(f_iter[i], graph_wrapped)-potential(f_iter[i+1], graph_wrapped) for i in range(len(f_iter)-1)]
plt.figure()
plt.title("Potential loss over time")
plt.ylabel("Potential loss")
plt.xlabel("No of iterations")
plt.plot(potential_loss)
plt.show()

plt.figure()
plt.title("Total time loss over time")
plt.ylabel("Total time loss")
plt.xlabel("No of iterations")
plt.plot(tt_iter)
plt.show()

## 10.Tests <a id="tests"></a>
***
Here are a bunch of tests to verify that our code is correct.

### 10.1 Test of the [travel time function](#travel_time)
- `flow_cost_solution(network) : optimal_flow, cost_solution`: takes as parameter a string which represent the path to the data file tntp `network` and return the flow and the cost of the STA solution by reading it in the data provided.
- `test_travel_time_function(network, graph)` tests if travel_time gives the correct cost when the optimal flow allocation is given in the data (like in __data/SiouxFalls/SiouxFalls__)

In [15]:
from network import flow_cost_solution

def test_travel_time_function(network, graph):
    """
    Parameter:
        file: a string which represent the path to the data file tntp
    Using the flow.tntp file (solutions) and the travel time function,
    the function tests if travel_time(flow_solutions) == cost_solutions
    Return:
        A vector of the differences between travel_time(flow_solutions) and cost_solutions for every links
    """
    flow, cost_solution = flow_cost_solution(network)
    print(np.linalg.norm(travel_time(flow, graph) - cost_solution))
    return travel_time(flow, graph) - cost_solution

# test if our travel_time function is correct
# network = 'data/SiouxFalls/SiouxFalls'
# network = 'data/Eastern-Massachusetts/EMA'
# network = 'data/Berlin-Tiergarten/berlin-tiergarten'
for network in {'data/SiouxFalls/SiouxFalls'}:
    graph_wrapped = init_travel_time_function(network)
    test_travel_time_function(network, graph_wrapped)
print()

3.277847697668423e-14



## References <a id="references"></a>
***
- [Patriksson] The Traffic Assignment Problem - Models and Methods, M.Patriksson